# Scrapping Data Twitter

In [4]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re
import time

In [24]:
# Path to the ChromeDriver
chrome_driver_path = 'chromedriver-win64/chromedriver.exe'

# Path to the Chrome binary
chrome_binary_path = 'chrome-win64/chrome.exe'

# Setting up Chrome options
chrome_options = Options()
chrome_options.binary_location = chrome_binary_path

In [25]:
# Initializing the ChromeDriver
service = Service(chrome_driver_path)
driver = webdriver.Chrome(service=service, options=chrome_options)

In [35]:
query = "chemical weapon"  # Replace with your search keyword
start_date = "2021-10-19"  # Start date for search
end_date = "2024-10-18"  # End date for search
url = f"https://twitter.com/search?q={query}%20since%3A{start_date}%20until%3A{end_date}&src=typed_query"

In [36]:
driver.get(url)

In [63]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import pandas as pd
import time

# Path to the ChromeDriver and Chrome binary
chrome_driver_path = 'chromedriver-win64/chromedriver.exe'  # Update with your actual path
chrome_binary_path = 'chrome-win64/chrome.exe'  # Update with your actual path

# Setting up Chrome options
chrome_options = Options()
chrome_options.binary_location = chrome_binary_path
# chrome_options.add_argument("--headless")  # Run in headless mode (no browser window)
# chrome_options.add_argument("--disable-gpu")
# chrome_options.add_argument("--no-sandbox")

# Initializing the ChromeDriver
service = Service(chrome_driver_path)
driver = webdriver.Chrome(service=service, options=chrome_options)

In [61]:
# Keywords to search for
keywords = [
    "battles"
]

# Date range for tweets
start_date = "2021-10-19"
end_date = "2024-10-18"

# Number of tweets to retrieve per keyword
tweet_limit_per_keyword = 10


# Function to scrape tweets for a given keyword
def scrape_tweets(keyword):
    # Build the Twitter search URL
    url = f"https://twitter.com/search?q={keyword}%20since%3A{start_date}%20until%3A{end_date}&src=typed_query"

    # Initialize the WebDriver and open the URL
#     driver = webdriver.Chrome(service=service, options=chrome_options)
    tweets = []
    try:
        driver.get(url)
        time.sleep(3)  # Initial wait to load the page

        # Initialize counters and status
        tweets_collected = 0  # Counter for tweets collected
        scroll_attempts = 0   # Number of scroll attempts
        last_height = driver.execute_script("return document.body.scrollHeight")  # Get the initial scroll height
        
        while tweets_collected < tweet_limit_per_keyword:
            # Parse the page source with BeautifulSoup
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            
            # Find each tweet container
            tweet_divs = soup.find_all('div', {'data-testid': 'tweetText'})
            for tweet_div in tweet_divs:
                try:
                    text = tweet_div.get_text()
                    
                    if text not in tweets:  # Avoid duplicates
                        tweets.append(text)
                        tweets_collected += 1
#                         print(f"Collected tweet {tweets_collected}: {text[:50]}...")  # Display a preview of each tweet

                    # Stop if we've reached the tweet limit for this keyword
                    if tweets_collected >= tweet_limit_per_keyword:
                        break

                except Exception as e:
                    time.sleep(2)
#                     print(f"An error occurred with a tweet: {e}")
                    continue  # Skip to the next tweet if an error occurs

            
            
            # Stop if we have collected enough tweets
            if tweets_collected >= tweet_limit_per_keyword:
                break
            
            # Scroll down to load more tweets
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(3)  # Wait to allow tweets to load

            # Check if the page height has changed to detect the end of the feed
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                time.sleep(2)
#                 print("No more new tweets loaded, stopping.")
                break  # Exit if we can't scroll further

            last_height = new_height  # Update the scroll height
            scroll_attempts += 1
#             print(f"Scrolling attempt {scroll_attempts}, tweets collected: {tweets_collected}")


            time.sleep(3)  # Wait to allow tweets to load

            # Additional short pause for anti-bot measures
            time.sleep(1)

    except Exception as e:
        time.sleep(2)
#         print(f"An error occurred while scraping {keyword}: {e}")

    finally:
        time.sleep(2)
#         driver.quit()  # Ensure the driver is closed properly
    
    return tweets

# Loop through each keyword and save tweets to a separate CSV file
for keyword in keywords:
    print(f"Scraping tweets for keyword: {keyword}")
    tweets = scrape_tweets(keyword)
    
    # Save tweets to a CSV file for each keyword
    df = pd.DataFrame(tweets, columns=["Tweet"])
    filename = f"tweets_{keyword.replace(' ', '_')}.csv"
    df.to_csv(filename, index=False)
    print(f"Saved {len(tweets)} tweets for keyword '{keyword}' to {filename}")

    # Pause between keyword searches to avoid triggering anti-bot mechanisms
    print(f"Pausing before the next keyword to avoid rate-limiting...")
    time.sleep(10)  # Longer pause between keyword searches

Scraping tweets for keyword: battles
Saved 10 tweets for keyword 'battles' to tweets_battles.csv
Pausing before the next keyword to avoid rate-limiting...


# FIX

In [ ]:
# Keywords to search for
keywords = [
    "battles", "explosions", "remote violence", "protests", "riots",
    "strategic developments", "violence against civilians",
    "looting", "property destruction", "peaceful protest", "air strike",
    "attack", "armed clash", "mob violence", "abduction", "disappearance",
    "IED", "sexual violence", "chemical weapon"
]

# Date range for tweets
start_date = "2021-10-19"
end_date = "2024-10-18"

# Number of tweets to retrieve per keyword
tweet_limit_per_keyword = 1000


# Function to scrape tweets for a given keyword
def scrape_tweets(keyword):
    # Build the Twitter search URL
    url = f"https://twitter.com/search?q={keyword}%20since%3A{start_date}%20until%3A{end_date}&src=typed_query"

    # Initialize the WebDriver and open the URL
#     driver = webdriver.Chrome(service=service, options=chrome_options)
    tweets = []
    try:
        driver.get(url)
        time.sleep(3)  # Initial wait to load the page

        # Initialize counters and status
        tweets_collected = 0  # Counter for tweets collected
        scroll_attempts = 0   # Number of scroll attempts
        max_scroll_attempts = 2  # Maximum scroll attempts if no new tweets load
        last_height = driver.execute_script("return document.body.scrollHeight")  # Get the initial scroll height
        
        while tweets_collected < tweet_limit_per_keyword:
            # Parse the page source with BeautifulSoup
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            
            # Find each tweet container
            tweet_divs = soup.find_all('article', {'data-testid': 'tweet'})
            for tweet_div in tweet_divs:
                try:
                    # Extract tweet text
                    text_div = tweet_div.find('div', {'data-testid': 'tweetText'})
                    text = text_div.get_text() if text_div else ""

                    # Extract tweet date
                    date_div = tweet_div.find('div', {'data-testid': 'User-Name'})
                    time_tag = date_div.find('time') if date_div else None
                    tweet_date = time_tag['datetime'][:10] if time_tag else ""  # Format date as "YYYY-MM-DD"

                    if text and tweet_date and {'text': text, 'date': tweet_date} not in tweets:  # Avoid duplicates
                        tweets.append({'text': text, 'date': tweet_date})
                        tweets_collected += 1
#                         print(f"Collected tweet {tweets_collected}: {text[:50]}... Date: {tweet_date}")  # Display tweet and date preview

                    # Stop if we've reached the tweet limit for this keyword
                    if tweets_collected >= tweet_limit_per_keyword:
                        break
                
                    time.sleep(2)
                    
                except Exception as e:
                    time.sleep(2)
#                     print(f"An error occurred with a tweet: {e}")
                    continue  # Skip to the next tweet if an error occurs

            
            
            # Stop if we have collected enough tweets
            if tweets_collected >= tweet_limit_per_keyword:
                break
            
            # Scroll down to load more tweets
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(3)  # Wait to allow tweets to load

            # Check if the page height has changed to detect the end of the feed
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                scroll_attempts += 1
#                 print(f"No new tweets loaded. Scroll attempt {scroll_attempts}/{max_scroll_attempts}")
                
                # Stop if maximum scroll attempts are reached
                if scroll_attempts >= max_scroll_attempts:
#                     print("Maximum scroll attempts reached. Moving to the next keyword.")
                    break
            else:
                # Reset scroll attempts if new tweets are loaded
                scroll_attempts = 0

            last_height = new_height  # Update the scroll height
            scroll_attempts += 1
#             print(f"Scrolling attempt {scroll_attempts}, tweets collected: {tweets_collected}")


            time.sleep(3)  # Wait to allow tweets to load

            # Additional short pause for anti-bot measures
            time.sleep(1)

    except Exception as e:
        time.sleep(2)
#         print(f"An error occurred while scraping {keyword}: {e}")

    finally:
        time.sleep(2)
#         driver.quit()  # Ensure the driver is closed properly
    
    return tweets

# Loop through each keyword and save tweets to a separate CSV file
for keyword in keywords:
    print(f"Scraping tweets for keyword: {keyword}")
    tweets = scrape_tweets(keyword)
    
    # Convert tweets to a DataFrame with 'Tweet' and 'Date' columns
    df = pd.DataFrame(tweets, columns=["text", "date"])
    
    # Rename columns to "Tweet" and "Date" for clarity in the CSV file
    df = df.rename(columns={"text": "Tweet", "date": "Date"})
    
    # Save the DataFrame to CSV
    filename = f"tweets_{keyword.replace(' ', '_')}.csv"
    df.to_csv(filename, index=False)
#     print(f"Saved {len(tweets)} tweets for keyword '{keyword}' to {filename}")

    # Pause between keyword searches to avoid rate-limiting
#     print("Pausing before the next keyword to avoid rate-limiting...")
    time.sleep(10)  # Longer pause between keyword searches



Scraping tweets for keyword: battles
Scraping tweets for keyword: explosions
Scraping tweets for keyword: remote violence
Scraping tweets for keyword: protests
Scraping tweets for keyword: riots
Scraping tweets for keyword: strategic developments
Scraping tweets for keyword: violence against civilians
Scraping tweets for keyword: looting
Scraping tweets for keyword: property destruction
Scraping tweets for keyword: peaceful protest
Scraping tweets for keyword: air strike


In [ ]:
# Keywords to search for
keywords = ['battle',
 'combat',
 'skirmish',
 'explosion',
 'blast',
 'detonation',
 'remote attack',
 'distance violence',
 'proxy violence',
 'protest',
 'demonstration',
 'rally',
 'riot',
 'civil unrest',
 'clashes',
 'strategic change',
 'policy shift',
 'geopolitical event',
 'civilian violence',
 'attacks on civilians',
 'civilian casualties',
 'theft',
 'plundering',
 'raiding',
 'vandalism',
 'damage to property',
 'infrastructure damage',
 'nonviolent protest',
 'peaceful demonstration',
 'peace rally',
 'airstrike',
 'air raid',
 'aerial attack',
 'assault',
 'strike',
 'offensive',
 'armed confrontation',
 'firefight',
 'armed encounter',
 'group violence',
 'crowd attack',
 'mass assault',
 'kidnapping',
 'capture',
 'hostage-taking',
 'missing person',
 'vanishing',
 'abduction case',
 'improvised explosive device',
 'bomb',
 'explosive',
 'sexual assault',
 'harassment',
 'abuse',
 'chemical attack',
 'toxic weapon',
 'poison gas',
 'military operation',
 'civil war',
 'genocide',
 'ethnic violence',
 'human rights violation',
 'martial law',
 'terrorist attack',
 'guerrilla warfare',
 'cyber attack',
 'human trafficking',
 'forced migration',
 'refugee crisis',
 'drone strike',
 'border conflict',
 'cross-border attack',
 'state violence',
 'mass shooting',
 'political unrest',
 'rebellion',
 'counter-insurgency']


# Date range for tweets
start_date = "2021-10-19"
end_date = "2024-10-18"

# Number of tweets to retrieve per keyword
tweet_limit_per_keyword = 1000


# Function to scrape tweets for a given keyword
def scrape_tweets(keyword):
    # Build the Twitter search URL
    url = f"https://twitter.com/search?q={keyword}%20since%3A{start_date}%20until%3A{end_date}&src=typed_query"

    # Initialize the WebDriver and open the URL
#     driver = webdriver.Chrome(service=service, options=chrome_options)
    tweets = []
    try:
        driver.get(url)
        time.sleep(3)  # Initial wait to load the page

        # Initialize counters and status
        tweets_collected = 0  # Counter for tweets collected
        scroll_attempts = 0   # Number of scroll attempts
        max_scroll_attempts = 2  # Maximum scroll attempts if no new tweets load
        last_height = driver.execute_script("return document.body.scrollHeight")  # Get the initial scroll height
        
        while tweets_collected < tweet_limit_per_keyword:
            # Parse the page source with BeautifulSoup
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            
            # Find each tweet container
            tweet_divs = soup.find_all('article', {'data-testid': 'tweet'})
            for tweet_div in tweet_divs:
                try:
                    # Extract tweet text
                    text_div = tweet_div.find('div', {'data-testid': 'tweetText'})
                    text = text_div.get_text() if text_div else ""

                    # Extract tweet date
                    date_div = tweet_div.find('div', {'data-testid': 'User-Name'})
                    time_tag = date_div.find('time') if date_div else None
                    tweet_date = time_tag['datetime'][:10] if time_tag else ""  # Format date as "YYYY-MM-DD"

                    if text and tweet_date and {'text': text, 'date': tweet_date} not in tweets:  # Avoid duplicates
                        tweets.append({'text': text, 'date': tweet_date})
                        tweets_collected += 1
#                         print(f"Collected tweet {tweets_collected}: {text[:50]}... Date: {tweet_date}")  # Display tweet and date preview

                    # Stop if we've reached the tweet limit for this keyword
                    if tweets_collected >= tweet_limit_per_keyword:
                        break
                
                    time.sleep(2)
                    
                except Exception as e:
                    time.sleep(2)
#                     print(f"An error occurred with a tweet: {e}")
                    continue  # Skip to the next tweet if an error occurs

            
            
            # Stop if we have collected enough tweets
            if tweets_collected >= tweet_limit_per_keyword:
                break
            
            # Scroll down to load more tweets
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(3)  # Wait to allow tweets to load

            # Check if the page height has changed to detect the end of the feed
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                scroll_attempts += 1
#                 print(f"No new tweets loaded. Scroll attempt {scroll_attempts}/{max_scroll_attempts}")
                
                # Stop if maximum scroll attempts are reached
                if scroll_attempts >= max_scroll_attempts:
#                     print("Maximum scroll attempts reached. Moving to the next keyword.")
                    break
            else:
                # Reset scroll attempts if new tweets are loaded
                scroll_attempts = 0

            last_height = new_height  # Update the scroll height
            scroll_attempts += 1
#             print(f"Scrolling attempt {scroll_attempts}, tweets collected: {tweets_collected}")


            time.sleep(3)  # Wait to allow tweets to load

            # Additional short pause for anti-bot measures
            time.sleep(1)

    except Exception as e:
        time.sleep(2)
#         print(f"An error occurred while scraping {keyword}: {e}")

    finally:
        time.sleep(2)
#         driver.quit()  # Ensure the driver is closed properly
    
    return tweets

# Loop through each keyword and save tweets to a separate CSV file
for keyword in keywords:
    print(f"Scraping tweets for keyword: {keyword}")
    tweets = scrape_tweets(keyword)
    
    # Convert tweets to a DataFrame with 'Tweet' and 'Date' columns
    df = pd.DataFrame(tweets, columns=["text", "date"])
    
    # Rename columns to "Tweet" and "Date" for clarity in the CSV file
    df = df.rename(columns={"text": "Tweet", "date": "Date"})
    
    # Save the DataFrame to CSV
    filename = f"tweets_{keyword.replace(' ', '_')}.csv"
    df.to_csv(filename, index=False)
#     print(f"Saved {len(tweets)} tweets for keyword '{keyword}' to {filename}")

    # Pause between keyword searches to avoid rate-limiting
#     print("Pausing before the next keyword to avoid rate-limiting...")
    time.sleep(10)  # Longer pause between keyword searches



Scraping tweets for keyword: battle
Scraping tweets for keyword: combat
Scraping tweets for keyword: skirmish
Scraping tweets for keyword: explosion
Scraping tweets for keyword: blast
Scraping tweets for keyword: detonation
Scraping tweets for keyword: remote attack
Scraping tweets for keyword: distance violence
Scraping tweets for keyword: proxy violence
Scraping tweets for keyword: protest
Scraping tweets for keyword: demonstration
Scraping tweets for keyword: rally
Scraping tweets for keyword: riot
Scraping tweets for keyword: civil unrest
Scraping tweets for keyword: clashes
Scraping tweets for keyword: strategic change
Scraping tweets for keyword: policy shift
Scraping tweets for keyword: geopolitical event
Scraping tweets for keyword: civilian violence
Scraping tweets for keyword: attacks on civilians
